# Part 1 : Data Preparation, Process, and Store Features

<a id='all-up-overview'></a>

## [Overview](./0-AutoClaimFraudDetection.ipynb)
* Notebook 1/2: Overview, Architecture and Data Exploration
* **[Notebook 3: Data Preparation, Process, and Store Features](./1-data-prep-e2e.ipynb)**
  * **[Architecture](#arch)**
  * **[Getting started](#aud-getting-started)**
  * **[DataSets](#aud-datasets)**
  * **[SageMaker Feature Store](#aud-feature-store)**
  * **[Create train and test datasets](#aud-dataset)**
* Notebook 4: Train, Check Bias, Tune, Record Lineage, and Register a Model
* Notebook 5: Mitigate Bias, Train New Model, Store in Registry)
* Notebook 6: Deploy Model, Run Predictions
* Notebook 7: Create and Run an End-to-End Pipeline to Deploy the Model

The purpose of this notebook is to perform the Data Prep phase of the ML life cycle. The main Data Wrangling, data ingestion, and multiple transformations will be done through the SageMaker Studio Data Wrangler GUI.

In this notebook, we will take the `.flow` files that define the transformations to the raw data. and apply them using a SageMaker Processing job that will apply those transformations to the raw data deposited in the S3 bucket as `.csv` files.

<a id='arch'> </a>

## Architecture for Data Prep, Process and Store Features
[overview](#all-up-overview)
----
![Data Prep and Store](./images/e2e-1-pipeline-v3b.png)

### Install required and/or update third-party libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')
!python -m pip install -Uq pip
!python -m pip install -q awswrangler imbalanced-learn
!python -m pip install --upgrade sagemaker

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils 

### Loading stored variables
If you ran this notebook before, you may want to re-use the resources you aready created with AWS. Run the cell below to load any prevously created variables. You should see a print-out of the existing variables. If you don't see anything printed then it's probably the first time you are running the notebook! 

In [3]:
%store -r
%store

Stored variables and their in-db values:
bucket                             -> 'sagemaker-us-east-1-875692608981'
claims_fg_name                     -> 'fraud-detect-demo-claims'
claims_preprocessed                ->       policy_id  incident_severity  num_vehicles_i
claims_table                       -> 'fraud-detect-demo-claims-1636518800'
clarify_expl_job_name              -> 'Clarify-Explainability-2021-11-10-14-35-21-747'
col_order                          -> ['fraud', 'num_injuries', 'incident_severity', 'in
customers_fg_name                  -> 'fraud-detect-demo-customers'
customers_preprocessed             ->       policy_id  customer_age  customer_education 
customers_table                    -> 'fraud-detect-demo-customers-1636518803'
database_name                      -> 'sagemaker_featurestore'
endpoint_config_name               -> 'fraud-detect-demo-xgboost-post-smote-endpoint-con
endpoint_name                      -> 'fraud-detect-demo-2021-11-11-08-19-15'
hyperparameter

**<font color='red'>Important</font>: You must have run the previous sequential notebooks to retrieve variables using the StoreMagic command.**

### Import libraries

In [4]:
import json
import time
import boto3
import string
import sagemaker
import pandas as pd
import awswrangler as wr

from sagemaker.feature_store.feature_group import FeatureGroup

<a id='aud-getting-started'></a>

## Getting started: Creating Resources

[overview](#all-up-overview)
----
In order to successfully run this notebook you will need to create some AWS resources. 
First, an S3 bucket will be created to store all the data for this tutorial. 
Once created, you will then need to create an AWS Glue role using the IAM console then attach a policy to the S3 bucket to allow FeatureStore access to this notebook. If you've already run this notebook and are picking up where you left off, then running the cells below should pick up the resources you already created without creating any additional resources.

#### Add FeatureStore policy to Studio's execution role

![title](images/iam-policies.png)


1. In a separate brower tab go to the IAM section of the AWS Console
2. Navigate to the Roles section and select the execution role you're using for your SageMaker Studio user
    * If you're not sure what role you're using, run the cell below to print it out
3. Attach the <font color='green'> AmazonSageMakerFeatureStoreAccess </font> policy to this role. Once attached, the changes take  effect immediately.

In [5]:
print("SageMaker Role:", sagemaker.get_execution_role().split("/")[-1])

SageMaker Role: AmazonSageMaker-ExecutionRoleMLT


### Set region, boto3 and SageMaker SDK variables

In [6]:
# You can change this to a region of your choice

region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

Using AWS Region: us-east-1


In [7]:
boto3.setup_default_session(region_name=region)

boto_session = boto3.Session(region_name=region)

s3_client = boto3.client("s3", region_name=region)

sagemaker_boto_client = boto_session.client("sagemaker")

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_boto_client
)

In [8]:
"""
Note: if you are not running this notebook from SageMaker Studio or SageMaker Classic Notebooks you will need to instanatiate 
the sagemaker_execution_role_name with an AWS role that has SageMakerFullAccess and SageMakerFeatureStoreFullAccess
"""
sagemaker_execution_role_name = "AmazonSageMaker-ExecutionRole-20210107T234882"
try:
    sagemaker_role = sagemaker.get_execution_role()
    print(f"\n sagemaker_role : {sagemaker_role}")
except ValueError:
    iam = boto3.client("iam")
    sagemaker_role = iam.get_role(RoleName=sagemaker_execution_role_name)["Role"]["Arn"]
    print(f"\n instantiating sagemaker_role with supplied role name : {sagemaker_role}")

account_id = boto3.client("sts").get_caller_identity()["Account"]


 sagemaker_role : arn:aws:iam::875692608981:role/AmazonSageMaker-ExecutionRoleMLT


### Create a directory in the SageMaker default bucket for this tutorial

In [9]:
if 'bucket' not in locals():
    bucket = sagemaker_session.default_bucket()
    prefix = 'fraud-detect-demo'
    %store bucket
    %store prefix
    print(f'Creating bucket: {bucket}...')

If you want to use your own S3 bucket that's already existing, uncomment and utilize the following example code.

In [10]:
"""
try:
    s3_client.create_bucket(Bucket=bucket, ACL='private', CreateBucketConfiguration={'LocationConstraint': region})
    print('Create S3 bucket: SUCCESS')
    
except Exception as e:
    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print(f'Using existing bucket: {bucket}/{prefix}')
    else:
        raise(e)
"""

"\ntry:\n    s3_client.create_bucket(Bucket=bucket, ACL='private', CreateBucketConfiguration={'LocationConstraint': region})\n    print('Create S3 bucket: SUCCESS')\n    \nexcept Exception as e:\n    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':\n        print(f'Using existing bucket: {bucket}/{prefix}')\n    else:\n        raise(e)\n"

In [11]:
# ======> Tons of output_paths
traing_job_output_path = f"s3://{bucket}/{prefix}/training_jobs"
bias_report_1_output_path = f"s3://{bucket}/{prefix}/clarify-bias-1"
bias_report_2_output_path = f"s3://{bucket}/{prefix}/clarify-bias-2"
explainability_output_path = f"s3://{bucket}/{prefix}/clarify-explainability"
processing_dir = "/opt/ml/processing"
claims_flow_uri = f"s3://{bucket}/{prefix}/flow/claims.flow"
customers_flow_uri = f"s3://{bucket}/{prefix}/flow/customers.flow"
train_data_uri = f"s3://{bucket}/{prefix}/data/train/train.csv"
test_data_uri = f"s3://{bucket}/{prefix}/data/test/test.csv"

# =======> variables used for parameterizing the notebook run
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"

claify_instance_count = 1
clairfy_instance_type = "ml.c5.xlarge"

predictor_instance_count = 1
predictor_instance_type = "ml.c5.xlarge"

### Upload raw data to S3
Before you can preprocess the raw data with Data Wrangler, it must exist in S3.

In [12]:
s3_client.upload_file(
    Filename="data/claims.csv", Bucket=bucket, Key=f"{prefix}/data/raw/claims.csv"
)
s3_client.upload_file(
    Filename="data/customers.csv", Bucket=bucket, Key=f"{prefix}/data/raw/customers.csv"
)

### Update attributes within the  `.flow` file 
DataWrangler will generate a .flow file. It contains a reference to an S3 bucket used during the Wrangling. This may be different from the one you have as a default in this notebook eg if the Wrangling was done by someone else, you will probably not have access to their bucket and you now need to point to your own S3 bucket so you can actually load the .flow file into Wrangler or access the data.

After running the cell below you can open the `claims.flow` and `customers.flow` files and export the data to S3 or you can continue the guide using the provided `data/claims_preprocessed.csv` and `data/customers_preprocessed.csv` files.

In [14]:
claims_flow_template_file = "flows/claims_flow_template"

with open(claims_flow_template_file, "r") as f:
    variables = {"bucket": bucket, "prefix": prefix}
    template = string.Template(f.read())
    claims_flow = template.substitute(variables)
    claims_flow = json.loads(claims_flow)

with open("flows/claims.flow", "w") as f:
    json.dump(claims_flow, f)

customers_flow_template_file = "flows/customers_flow_template"

with open(customers_flow_template_file, "r") as f:
    variables = {"bucket": bucket, "prefix": prefix}
    template = string.Template(f.read())
    customers_flow = template.substitute(variables)
    customers_flow = json.loads(customers_flow)

with open("flows/customers.flow", "w") as f:
    json.dump(customers_flow, f)

### Upload raw data to S3

In [16]:
s3_client.upload_file(
    Filename="flows/claims.flow", Bucket=bucket, Key=f"{prefix}/flow/claims.flow"
)
s3_client.upload_file(
    Filename="flows/customers.flow", Bucket=bucket, Key=f"{prefix}/flow/customers.flow"
)

----

### Next Notebook: [Train, Check Bias, Tune, Record Lineage, Register Model](./04-Lineage-Train-Assess-Bias-Tune-Registry.ipynb)

In [ ]:
train_data_uri